In [94]:
# IMPORTS & DEFINITIONS

import csv, sys
import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np; np.set_printoptions(threshold=sys.maxsize)
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
import math
from sklearn.model_selection import cross_val_score

DUD_VALUE= 0 # change from 0 to something like 123 for debugging
TOTAL_ROWS= 2228278 # This number is greater than the total amount of rows circa epoch change to 27th, but it still works
INPUT_ROWS_LIMIT= TOTAL_ROWS
FILENAME= 'dublinbikes_2020_Q1.csv'
MAX_STATION_ID= 117
SECS_IN_5MIN= 300
DATAPOINT_EVERYX_MIN= 5
DATAPOINTS_PER_DAY= 288
DAYS_OF_WEEK= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'] # yes, I consider Monday to be the '0'/start of the week
STARTING_DAY= 0 # aka Monday. Because the 27th of Jan 2020 is a Monday
MISSING_STATIONS= [117, 116, 70, 60, 46, 35, 20, 14, 1]
TOTAL_DAYS= 66 # from 27 / 1 / 2020 to (and including) 1 / 4 / 2020
HOURS= 24
EPOCH= datetime.datetime(2020, 1, 27, 0, 0)
MAX_TIME= int((datetime.datetime(2020,4,2,0,0) - EPOCH).total_seconds() / SECS_IN_5MIN)
K= 5
STEP_SIZE= 0.02185 # just the magic number that leads to 288 values being generated
R= 0.5
MAX_HINDSIGHT= 60

class DataDay: # ideally this would be nested in the Station class
    def __init__(self, index):
        self.index= index
        self.times_populated= 0
        self.day_of_week= DUD_VALUE
        self.day_since_epoch= DUD_VALUE
        self.day_of_week= ((STARTING_DAY + index - 1) % len(DAYS_OF_WEEK))
        
        self.daily_epoch_time= np.full(DATAPOINTS_PER_DAY, DUD_VALUE, dtype=np.int)
        self.epoch_time= np.full(DATAPOINTS_PER_DAY, DUD_VALUE, dtype=np.int)
        self.bikes= np.full(DATAPOINTS_PER_DAY, DUD_VALUE, dtype=np.int)
        self.percent_bikes= np.full(DATAPOINTS_PER_DAY, DUD_VALUE, dtype=np.float)

    def populate(self, daily_epoch_time, epoch_time, bikes, percent_bikes):
        self.daily_epoch_time[daily_epoch_time]= daily_epoch_time
        self.epoch_time[daily_epoch_time]= epoch_time
        self.bikes[daily_epoch_time]= bikes
        self.percent_bikes[daily_epoch_time]= percent_bikes
        self.times_populated+= 1

class Station:
    def __init__(self, index):
        self.index= index
        self.name= DUD_VALUE
        self.bike_capacity= DUD_VALUE
        self.address= DUD_VALUE
        self.latitude= DUD_VALUE
        self.longitude= DUD_VALUE
        self.data_days= [DataDay(i) for i in range(1, TOTAL_DAYS + 2)] # + 1 because 0 is excluded and + 1 because we want to include the last day, not have it as the rim
    
    def populate_consts(self, name, bike_capacity, address, latitude, longitude):
        self.name= name
        self.bike_capacity= bike_capacity
        self.address= address
        self.latitude= latitude
        self.longitude= longitude

def get_station_id(name):
    try:
        index= [x.name for x in stations].index(name)
    except ValueError:
        index= -1
    return index

In [2]:
# DATA STRUCTURING

total_capacity= 0 # not in use currently
index= []; daily_epoch_time= []; epoch_time= []; percent_bikes= [];
stations= [Station(i) for i in range(1, MAX_STATION_ID + 1)] # note: MAX_STATION_ID + 1 is not included in the range
indices_to_populate= list(range(1, MAX_STATION_ID + 1))
for index in MISSING_STATIONS:
    indices_to_populate.remove(index)

with open(FILENAME, newline='') as f:
    reader = csv.reader(f); next(reader) # skip data header
    current_index= 0
    try:
        while len(indices_to_populate) != 0:
            row= next(reader)
            if int(row[0]) == current_index: # this 'if' is just for performance
                continue
            current_index= int(row[0])
            if current_index in indices_to_populate:
                stations[current_index - 2].populate_consts(row[3], row[4], row[8], row[9], row[10])
                indices_to_populate.remove(current_index)
                total_capacity+= int(row[4])
        
        f.seek(0)
        reader= csv.reader(f); row= next(reader) # skip data header
        for row_i, row in enumerate(reader):
            if row_i >= INPUT_ROWS_LIMIT:
                break
            if int((datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), int(row[1][11: 13]), int(row[1][14: 16])) - EPOCH).total_seconds()) < 0:
                continue
            try:
                epoch_time= int((datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), int(row[1][11: 13]), int(row[1][14: 16])) - EPOCH).total_seconds() / SECS_IN_5MIN)
                stations[int(row[0]) - 1].data_days[int(epoch_time / (DATAPOINTS_PER_DAY - 1))].populate( \
                    int((datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), int(row[1][11: 13]), int(row[1][14: 16])) - datetime.datetime(int(row[1][0:4]), int(row[1][5:7]), int(row[1][8:10]), 0, 0)).total_seconds() / (SECS_IN_5MIN + 1)), \
                    epoch_time, \
                    int(row[6]), \
                    float("{:.3f}".format(float(row[6]) / float(row[4]))))
            except IndexError as e:
                print("\nTRIED: ", epoch_time, ' / ', DATAPOINTS_PER_DAY, ' = ', int(epoch_time / (DATAPOINTS_PER_DAY - 1)))
                print(row[1])
    except csv.Error as e:
        sys.exit('file {}, line {}: {}'.format(filename, reader.line_num, e))

In [129]:
# FEATURE DATA PREPERATION

fullness= np.full((MAX_TIME, MAX_STATION_ID - len(MISSING_STATIONS)), DUD_VALUE, dtype=np.int)
fullness_in10= np.full((MAX_TIME, MAX_STATION_ID - len(MISSING_STATIONS)), DUD_VALUE, dtype=np.int)
fullness_in30= np.full((MAX_TIME, MAX_STATION_ID - len(MISSING_STATIONS)), DUD_VALUE, dtype=np.int)
fullness_in60= np.full((MAX_TIME, MAX_STATION_ID - len(MISSING_STATIONS)), DUD_VALUE, dtype=np.int)
fullness_percent= np.full((MAX_TIME, MAX_STATION_ID - len(MISSING_STATIONS)), DUD_VALUE, dtype=np.float)
bikes_changes_pastx= np.full((MAX_TIME, MAX_STATION_ID - len(MISSING_STATIONS), int(MAX_HINDSIGHT / DATAPOINT_EVERYX_MIN)), DUD_VALUE, dtype=np.int)
day_of_week= np.full((MAX_TIME, len(DAYS_OF_WEEK)), DUD_VALUE, dtype=np.int)
hour_of_day= np.full((MAX_TIME, HOURS), DUD_VALUE, dtype=np.float)

station_index_decrement= 1 # this is a varying offset for the indexing of stations that accounts for missing stations that are being ignored
for epoch_day_i in range(TOTAL_DAYS):
    print("########### epoch_day_i: ", epoch_day_i)
    x_offset= epoch_day_i * DATAPOINTS_PER_DAY
    y_offset= 0
    
    block= np.zeros((DATAPOINTS_PER_DAY, HOURS), dtype=np.float)
    daily_epoch_time= list(range(DATAPOINTS_PER_DAY))
    for time_i in daily_epoch_time:
        hour= float("{:.3f}".format(time_i / 12))
        block[time_i][(int(hour) + 1) % HOURS]= hour % 1
        block[time_i][int(hour)]= 1 - (hour % 1)
    hour_of_day[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
    
    day= stations[2].data_days[epoch_day_i].day_of_week
    block= np.zeros((DATAPOINTS_PER_DAY, len(DAYS_OF_WEEK)), dtype=np.int)
    for block_i, sub_arr in enumerate(block):
        block[block_i][day]= 1
    day_of_week[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
    
    for station in stations:
        print("###### station.index: ", station.index)
        if station.index == 1:
            station_index_decrement= 1
        if station.index in MISSING_STATIONS:
            station_index_decrement+= 1
            continue
        y_offset= station.index - station_index_decrement
        
        block= station.data_days[epoch_day_i].percent_bikes
        block= np.reshape(block, (DATAPOINTS_PER_DAY, 1))
        fullness_percent[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        
        block= station.data_days[epoch_day_i].bikes
        block= np.reshape(block, (DATAPOINTS_PER_DAY, 1))
        fullness[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        
        bikes= station.data_days[epoch_day_i].bikes
        block= np.reshape(bikes[2:], (bikes.shape[0] - 2, 1))
        fullness_in10[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        block= np.reshape(bikes[6:], (bikes.shape[0] - 6, 1))
        fullness_in30[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        block= np.reshape(bikes[12:], (bikes.shape[0] - 12, 1))
        fullness_in60[x_offset:x_offset + block.shape[0], y_offset:y_offset + block.shape[1]]= block
        
        block= np.reshape(station.data_days[epoch_day_i].bikes, (DATAPOINTS_PER_DAY, 1))
        block_xminchange= np.zeros((DATAPOINTS_PER_DAY, int(MAX_HINDSIGHT / DATAPOINT_EVERYX_MIN)), dtype=np.int)
        fullness_xago= np.zeros((DATAPOINTS_PER_DAY, int(MAX_HINDSIGHT / DATAPOINT_EVERYX_MIN)), dtype=np.int)
        for col_i in range(fullness_xago.shape[1]):
            fullness_xago[0:DATAPOINTS_PER_DAY, col_i:col_i + 1]= block # this is a provisional assignment; it will be overwritten if the right conditions are present
        
        block_xminchange[block_i:block_i + 1, 0:block_xminchange.shape[1]]= current - fullness_xago[block_i:block_i + 1, 0:block_xminchange.shape[1]]
        bikes_changes_pastx[x_offset:x_offset + block_xminchange.shape[0], y_offset:y_offset + 1, 0:block_xminchange.shape[1]]= np.reshape(block_xminchange, (DATAPOINTS_PER_DAY, 1, block_xminchange.shape[1]))

########### epoch_day_i:  0
###### station.index:  1
###### station.index:  2
###### station.index:  3
###### station.index:  4
###### station.index:  5
###### station.index:  6
###### station.index:  7
###### station.index:  8
###### station.index:  9
###### station.index:  10
###### station.index:  11
###### station.index:  12
###### station.index:  13
###### station.index:  14
###### station.index:  15
###### station.index:  16
###### station.index:  17
###### station.index:  18
###### station.index:  19
###### station.index:  20
###### station.index:  21
###### station.index:  22
###### station.index:  23
###### station.index:  24
###### station.index:  25
###### station.index:  26
###### station.index:  27
###### station.index:  28
###### station.index:  29
###### station.index:  30
###### station.index:  31
###### station.index:  32
###### station.index:  33
###### station.index:  34
###### station.index:  35
###### station.index:  36
###### station.index:  37
###### station.inde

###### station.index:  26
###### station.index:  27
###### station.index:  28
###### station.index:  29
###### station.index:  30
###### station.index:  31
###### station.index:  32
###### station.index:  33
###### station.index:  34
###### station.index:  35
###### station.index:  36
###### station.index:  37
###### station.index:  38
###### station.index:  39
###### station.index:  40
###### station.index:  41
###### station.index:  42
###### station.index:  43
###### station.index:  44
###### station.index:  45
###### station.index:  46
###### station.index:  47
###### station.index:  48
###### station.index:  49
###### station.index:  50
###### station.index:  51
###### station.index:  52
###### station.index:  53
###### station.index:  54
###### station.index:  55
###### station.index:  56
###### station.index:  57
###### station.index:  58
###### station.index:  59
###### station.index:  60
###### station.index:  61
###### station.index:  62
###### station.index:  63
###### stati

###### station.index:  81
###### station.index:  82
###### station.index:  83
###### station.index:  84
###### station.index:  85
###### station.index:  86
###### station.index:  87
###### station.index:  88
###### station.index:  89
###### station.index:  90
###### station.index:  91
###### station.index:  92
###### station.index:  93
###### station.index:  94
###### station.index:  95
###### station.index:  96
###### station.index:  97
###### station.index:  98
###### station.index:  99
###### station.index:  100
###### station.index:  101
###### station.index:  102
###### station.index:  103
###### station.index:  104
###### station.index:  105
###### station.index:  106
###### station.index:  107
###### station.index:  108
###### station.index:  109
###### station.index:  110
###### station.index:  111
###### station.index:  112
###### station.index:  113
###### station.index:  114
###### station.index:  115
###### station.index:  116
###### station.index:  117
########### epoch_da

###### station.index:  100
###### station.index:  101
###### station.index:  102
###### station.index:  103
###### station.index:  104
###### station.index:  105
###### station.index:  106
###### station.index:  107
###### station.index:  108
###### station.index:  109
###### station.index:  110
###### station.index:  111
###### station.index:  112
###### station.index:  113
###### station.index:  114
###### station.index:  115
###### station.index:  116
###### station.index:  117
########### epoch_day_i:  13
###### station.index:  1
###### station.index:  2
###### station.index:  3
###### station.index:  4
###### station.index:  5
###### station.index:  6
###### station.index:  7
###### station.index:  8
###### station.index:  9
###### station.index:  10
###### station.index:  11
###### station.index:  12
###### station.index:  13
###### station.index:  14
###### station.index:  15
###### station.index:  16
###### station.index:  17
###### station.index:  18
###### station.index:  19


###### station.index:  103
###### station.index:  104
###### station.index:  105
###### station.index:  106
###### station.index:  107
###### station.index:  108
###### station.index:  109
###### station.index:  110
###### station.index:  111
###### station.index:  112
###### station.index:  113
###### station.index:  114
###### station.index:  115
###### station.index:  116
###### station.index:  117
########### epoch_day_i:  16
###### station.index:  1
###### station.index:  2
###### station.index:  3
###### station.index:  4
###### station.index:  5
###### station.index:  6
###### station.index:  7
###### station.index:  8
###### station.index:  9
###### station.index:  10
###### station.index:  11
###### station.index:  12
###### station.index:  13
###### station.index:  14
###### station.index:  15
###### station.index:  16
###### station.index:  17
###### station.index:  18
###### station.index:  19
###### station.index:  20
###### station.index:  21
###### station.index:  22
###

...

###### station.index:  5
###### station.index:  6
###### station.index:  7
###### station.index:  8
###### station.index:  9
###### station.index:  10
###### station.index:  11
###### station.index:  12
###### station.index:  13
###### station.index:  14
###### station.index:  15
###### station.index:  16
###### station.index:  17
###### station.index:  18
###### station.index:  19
###### station.index:  20
###### station.index:  21
###### station.index:  22
###### station.index:  23
###### station.index:  24
###### station.index:  25
###### station.index:  26
###### station.index:  27
###### station.index:  28
###### station.index:  29
###### station.index:  30
###### station.index:  31
###### station.index:  32
###### station.index:  33
###### station.index:  34
###### station.index:  35
###### station.index:  36
###### station.index:  37
###### station.index:  38
###### station.index:  39
###### station.index:  40
###### station.index:  41
###### station.index:  42
###### station.in

###### station.index:  53
###### station.index:  54
###### station.index:  55
###### station.index:  56
###### station.index:  57
###### station.index:  58
###### station.index:  59
###### station.index:  60
###### station.index:  61
###### station.index:  62
###### station.index:  63
###### station.index:  64
###### station.index:  65
###### station.index:  66
###### station.index:  67
###### station.index:  68
###### station.index:  69
###### station.index:  70
###### station.index:  71
###### station.index:  72
###### station.index:  73
###### station.index:  74
###### station.index:  75
###### station.index:  76
###### station.index:  77
###### station.index:  78
###### station.index:  79
###### station.index:  80
###### station.index:  81
###### station.index:  82
###### station.index:  83
###### station.index:  84
###### station.index:  85
###### station.index:  86
###### station.index:  87
###### station.index:  88
###### station.index:  89
###### station.index:  90
###### stati

###### station.index:  1
###### station.index:  2
###### station.index:  3
###### station.index:  4
###### station.index:  5
###### station.index:  6
###### station.index:  7
###### station.index:  8
###### station.index:  9
###### station.index:  10
###### station.index:  11
###### station.index:  12
###### station.index:  13
###### station.index:  14
###### station.index:  15
###### station.index:  16
###### station.index:  17
###### station.index:  18
###### station.index:  19
###### station.index:  20
###### station.index:  21
###### station.index:  22
###### station.index:  23
###### station.index:  24
###### station.index:  25
###### station.index:  26
###### station.index:  27
###### station.index:  28
###### station.index:  29
###### station.index:  30
###### station.index:  31
###### station.index:  32
###### station.index:  33
###### station.index:  34
###### station.index:  35
###### station.index:  36
###### station.index:  37
###### station.index:  38
###### station.index:

###### station.index:  100
###### station.index:  101
###### station.index:  102
###### station.index:  103
###### station.index:  104
###### station.index:  105
###### station.index:  106
###### station.index:  107
###### station.index:  108
###### station.index:  109
###### station.index:  110
###### station.index:  111
###### station.index:  112
###### station.index:  113
###### station.index:  114
###### station.index:  115
###### station.index:  116
###### station.index:  117
########### epoch_day_i:  61
###### station.index:  1
###### station.index:  2
###### station.index:  3
###### station.index:  4
###### station.index:  5
###### station.index:  6
###### station.index:  7
###### station.index:  8
###### station.index:  9
###### station.index:  10
###### station.index:  11
###### station.index:  12
###### station.index:  13
###### station.index:  14
###### station.index:  15
###### station.index:  16
###### station.index:  17
###### station.index:  18
###### station.index:  19


###### station.index:  53
###### station.index:  54
###### station.index:  55
###### station.index:  56
###### station.index:  57
###### station.index:  58
###### station.index:  59
###### station.index:  60
###### station.index:  61
###### station.index:  62
###### station.index:  63
###### station.index:  64
###### station.index:  65
###### station.index:  66
###### station.index:  67
###### station.index:  68
###### station.index:  69
###### station.index:  70
###### station.index:  71
###### station.index:  72
###### station.index:  73
###### station.index:  74
###### station.index:  75
###### station.index:  76
###### station.index:  77
###### station.index:  78
###### station.index:  79
###### station.index:  80
###### station.index:  81
###### station.index:  82
###### station.index:  83
###### station.index:  84
###### station.index:  85
###### station.index:  86
###### station.index:  87
###### station.index:  88
###### station.index:  89
###### station.index:  90
###### stati

In [130]:
print(bikes_changes_pastx.shape)
print(bikes_changes_pastx[5000:5005])



(19008, 108, 12)
[[[0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0]
  [

In [117]:
# APPROACH DEFINITIONS

def run_approach1(station_name):
    index= get_station_id(station_name)
    
    y= np.full((MAX_TIME, 3), 0, dtype=np.int) # change the 3 to a 6 to do both stations at once on the generalised-training form of an approach
    y[0:MAX_TIME, 0:1]= np.reshape(fullness_in10[:,index], (MAX_TIME, 1))
    y[0:MAX_TIME, 1:2]= np.reshape(fullness_in30[:,index], (MAX_TIME, 1))
    y[0:MAX_TIME, 2:3]= np.reshape(fullness_in60[:,index], (MAX_TIME, 1))
    #y[0:MAX_TIME, 3:4]= np.reshape(fullness_in10[:,get_station_id("CUSTOM HOUSE QUAY")], (MAX_TIME, 1))
    #y[0:MAX_TIME, 4:5]= np.reshape(fullness_in30[:,get_station_id("CUSTOM HOUSE QUAY")], (MAX_TIME, 1))
    #y[0:MAX_TIME, 5:6]= np.reshape(fullness_in60[:,get_station_id("CUSTOM HOUSE QUAY")], (MAX_TIME, 1))

    X= np.full((MAX_TIME, hour_of_day.shape[1] + day_of_week.shape[1] + 4 \
                #* bikes_changes_past5.shape[1] \
               ), 0, dtype=np.float)
    X[0:MAX_TIME, 0:7]= day_of_week
    X[0:MAX_TIME, 7:31]= hour_of_day
    X[0:MAX_TIME, 31:32]= fullness_percent[0:MAX_TIME, index:index+1]
    X[0:MAX_TIME, 32:33]= bikes_changes_pastx[0:MAX_TIME, index:index+1, 0:1] # past5
    X[0:MAX_TIME, 33:34]= bikes_changes_pastx[0:MAX_TIME, index:index+1, 2:3] # past15
    X[0:MAX_TIME, 34:35]= bikes_changes_pastx[0:MAX_TIME, index:index+1, 8:9] # past45
    # X[0:MAX_TIME, 31:139]= fullness_percent
    # X[0:MAX_TIME, 139:247]= bikes_changes_past5
    # X[0:MAX_TIME, 247:355]= bikes_changes_past15
    # X[0:MAX_TIME, 355:463]= bikes_changes_past45


    kf= KFold(n_splits= K)
    kf.get_n_splits(X)
    score_sum= 0.0
    i= 1
    for train_index, test_index in kf.split(X):
        X_train, X_test= X[train_index], X[test_index]
        y_train, y_test= y[train_index], y[test_index]
        regr= MLPRegressor(random_state= 1, max_iter= 1000).fit(X_train, y_train)
        y_pred= regr.predict(X_test)
        score_sum+= regr.score(X_test, y_test)
        print("Data split ", i, " accuracy: ", regr.score(X_test, y_test) * 100, " %")
        i+= 1
    print("\nAverage accuracy of model: ", (score_sum / K) * 100, " %")

    # Below is alternative evaluation code

    # regr= MLPRegressor(random_state= 1, max_iter= 500).fit(X_train, y_train)
    # scores= cross_val_score(regr, X, y, cv=5)
    # print(scores)
    
def run_approach2(station_name):
    X= np.full((MAX_TIME, 2 + 3 \
            #* bikes_changes_past5.shape[1] \
           ), -1, dtype=np.float)
    
    positions= []; t= 0
    while t < 2 * math.pi:
        positions.append((1 - (R * math.cos(t) + R), R * math.sin(t) + R))
        t+= STEP_SIZE
    pos_i= 0
    for time_i in range(MAX_TIME):
        X[time_i, 0]= positions[pos_i][0]
        X[time_i, 1]= positions[pos_i][1]
        pos_i= (pos_i + 1) % len(positions)
    
    index= get_station_id(station_name)
    np.reshape
    X[0:MAX_TIME, 2:3]= np.reshape((bikes_changes_pastx[0:MAX_TIME, index:index+1, 0:1]), (MAX_TIME, 1)) # past5
    X[0:MAX_TIME, 3:4]= np.reshape((bikes_changes_pastx[0:MAX_TIME, index:index+1, 2:3]), (MAX_TIME, 1)) # past15
    X[0:MAX_TIME, 4:5]= np.reshape((bikes_changes_pastx[0:MAX_TIME, index:index+1, 8:9]), (MAX_TIME, 1)) # past45
    # X[0:MAX_TIME, 2:110]= bikes_changes_past5
    # X[0:MAX_TIME, 110:218]= bikes_changes_past15
    # X[0:MAX_TIME, 218:326]= fullness_percent
    
    y= np.full((MAX_TIME, 3), 0, dtype=np.int)
    y[0:MAX_TIME, 0:1]= np.reshape(fullness_in10[:,index], (MAX_TIME, 1))
    y[0:MAX_TIME, 1:2]= np.reshape(fullness_in30[:,index], (MAX_TIME, 1))
    y[0:MAX_TIME, 2:3]= np.reshape(fullness_in60[:,index], (MAX_TIME, 1))
    
    neigh= KNeighborsRegressor(n_neighbors= 30, weights='distance')
    cv_scores= cross_val_score(neigh, X, y, cv=5)
    print(cv_scores) # print each cv score (accuracy) and average them
    print('cv_scores mean:{}'.format(np.mean(cv_scores)))

In [118]:
# DRIVER

run_approach2("PORTOBELLO ROAD")
run_approach2("CUSTOM HOUSE QUAY")

[ 0.89566451  0.89295721  0.93357822  0.75157559 -2.85592423]
cv_scores mean:0.12357025844386715
[ 0.24633268  0.15837656  0.215309   -0.06393942 -0.65474032]
cv_scores mean:-0.019732301427142794
